# Simulation command generation from simulation sheet
>'It's turtles all the way down!'

Plan: 
- Have array of sample settings, maybe optimized to give comparable $\sigma t$ to give comparable signal strength.

In [ ]:
from definitions import s_t
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 3e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(2.165, 0.02165), (4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.02

delta_rho = 2e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: sigma * t: {st}, permissible: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

In [ ]:
from definitions import s_t


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.015

delta_rho = 1.8e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: ")
        print(f"\t\ts*t: {st}\n\t\t0.1 <= s*t <= 0.8: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        # print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

In [ ]:
print("{:,.3e}".format(np.sqrt(6.1e-7) * 1e18))

In [ ]:
from definitions import *


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
Rs = [2e-6, 300e-9, 50e-9]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]

# Source sample (thickness) pairing (L0, DL, R, t)
source_sample = [(4.321, 0.04321, 2e-6, 1e-3), (8, 0.8, 2e-6, 1e-3), (4.321, 0.04321, 300e-9, 10e-3), (8, 0.8, 300e-9, 5e-3), (4.321, 0.04321, 50e-9, 10e-3), (8, 0.8,50e-9, 10e-3)]

thickness_map = {
    (4.321, 2e-6):  1e-3,
    (8, 2e-6):  1e-3,
    (4.321, 300e-9): 10e-3,
    (8, 300e-9): 5e-3,
    (4.321, 50e-9): 10e-3,
    (8, 50e-9): 10e-3,
}
# Universal sample parameters (?)
phi = 0.015

delta_rho = 1.8e14 # 1/m^2 (?)
print(f"Constant parameters: drho = {delta_rho * 1e-14}e14 m^-2; phi = {phi}")
for (L0, DL, R, t) in source_sample:
    # print(thickness_map[(L0, R)])
    print(F"R = {R * 1e9}nm, t = {round(t * 1e3,2)}mm, lambda = {L0} Å:")
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    permissible = st < 0.8 and st >= 0.1
    print(f"\ts*t: {round(st, 4)}\n\t0.1 <= s*t <= 0.8: {permissible}")
    sigma = s_t(R,t, L0 * 1e-10) / t
    t_optimum = 0.38 / sigma

In [ ]:
import pandas as pd
import os 
df = pd.read_csv('simulations_new.csv', sep=',', header=0)

FWHM_env_min = 2e-3
# L_s_reduction = 0.75
# For simulating variable L_s
L_s_min = 1.5 
class Instrument:
    def __init__(self, id: str,  prec_type: str, L0: float, DL: float, theta_0: float, By_min: float, By_max: float, L_s: float = 1.8, L_1: float = 4, L_2: float = 2):
        self.id = id
        self.prec_type = prec_type
        self.L0 = L0
        self.DL = DL
        self.theta_0 = theta_0
        self.By_min = By_min 
        self.By_max = By_max
        self.L_s = L_s
        self.L_1 = L_1
        self.L_2 = L_2

    # Minimum reachable delta as follows from B field. 
    def delta_min_B_field(self):
        return compute_z(self.By_min * abs(1 - self.L_1 / self.L_2),self.theta_0,self.L0,L_s_min)
    
    # Limit to have at least 1 modulation period on the detector
    def delta_min_detector(self):
        f_min = 1/detector_size
        delta_min_sampling = f_min * self.L0 * L_s_min
        return delta_min_sampling

    def delta_min_named(self):
        delta_min_field = self.delta_min_B_field()
        delta_min_single_period = self.delta_min_detector()
        mins = [(delta_min_field, 'B strength'), (delta_min_single_period, 'detector size')]
        (d_min, min_name) = max(mins)
        return (d_min, min_name)
    
    def delta_min(self):
        return self.delta_min_named()[0]

    def delta_max_named(self):
        d_max_field = self.delta_max_B_field()
        delta_max_env = self.delta_max_envelope()
        delta_max_ten_samples = self.delta_max_sampling()
        maxes = [(d_max_field, 'B strength'), (delta_max_env, 'envelope'), (delta_max_ten_samples, 'sampling')]
        (d_max, max_name) = min(maxes)
        return (d_max, max_name) 

    def delta_max(self):
        return self.delta_max_named()[0]
    
    def delta_max_B_field(self):
        # Due to the focussing condition, one component will have field By_max/2 and the other By_max, giving a delta of By_max/2
        return compute_z(self.By_max * (1 - self.L_2 / self.L_1), self.theta_0, self.L0, self.L_s)

    def delta_max_sampling(self, samples = 10):
        f_s = 1/detector_pixel_size
        f_super_sampled = f_s / samples
        delta_max_sampling = f_super_sampled * self.L0 * self.L_s
        return delta_max_sampling

    def delta_max_envelope(self):
        delta_max_env = np.sqrt(2 * np.log(2)) * self.L0**2 * self.L_s / (np.pi * self.DL * FWHM_env_min)
        return delta_max_env
    
    def delta_range(self):
        return self.delta_min(), self.delta_max()

    def prec_type_long(self):
        long_str = ''
        match self.prec_type:
            case 'iso':  
                long_str = 'isosceles triangle'
            case 'wsp':
                long_str = 'Wollaston prism'
            case 'foil':
                long_str = 'foil flipper'
        return long_str

            
    def __str__(self):
        d_max_field = self.delta_max_B_field()
        delta_max_env = self.delta_max_envelope()
        delta_max_ten_samples = self.delta_max_sampling()
        # print(F"Max delta ideal sampling (10 samples per period) (f_0 = {round(f_ten_samples*1e-3)}mm^-1: {round(delta_max_ten_samples * 1e9,2)}nm")
        maxes = [(d_max_field, 'precession devices'), (delta_max_env, 'envelope'), (delta_max_ten_samples, 'sampling')]
        (d_max, max_name) = min(maxes)
        delta_min_field = self.delta_min_B_field()
        delta_min_single_period = self.delta_min_detector()
        mins = [(delta_min_field, 'precession devices'), (delta_min_single_period, 'detector size')]
        (d_min, min_name) = max(mins)
        # print(min_max)
        return f"""Instrument ID {self.id}
    \tSource: L0 = {self.L0 * 1e10} Å; sigma_L = {self.DL * 1e10} Å
    \tPrecession device: {self.prec_type_long()}; theta_0 = {round(self.theta_0,2)} rad; By_min = {self.By_min * 1e3}mT; By_max = {self.By_max * 1e3}mT
    \tL_1 = {round(self.L_1, 2)}m; L_2 = {round(self.L_2, 2)}m;  L_s = {round(self.L_s, 2)}m
    \tδ range from precession devices: {round(delta_min_field * 1e9,1)} - {round(d_max_field * 1e9, 1)}nm
    \tmin δ for 1 period on detector with height {detector_size * 1e3}mm: {round(d_min * 1e9,1)}nm 
    \tmax δ for sampling at 10s/period (f_0 = {round(f_s / 10 *1e-3)}mm^-1): {round(delta_max_ten_samples * 1e9,1)}nm
    \tmax δ for envelope FWHM due to wavelength spread >= {FWHM_env_min*1e3}mm: {round(delta_max_env * 1e9,1)}nm
    \tfinal δ range: {round(d_min * 1e9,1)} - {round(d_max * 1e9, 1)}nm ({min_name} - {max_name} limited)"""
    

# print(df.values)
instrs = []
for r in df.values:
    # print(r)
    id = r[0]
    L0 = float(r[1]) * 1e-10
    DL = float(r[2]) * 1e-10 / FWHM_factor
    prec = r[4]
    theta_0 = np.deg2rad(float(r[5]))
    By_min = float(r[6]) * 1e-3
    By_max = float(r[7]) * 1e-3
    instr = Instrument(id, prec, L0, DL, theta_0, By_min, By_max)
    instrs.append(instr)

for instr in instrs:
    print(str(instr))

In [ ]:
N = 1000000
N_steps = 30
def find_overlap(interval1, interval2):
    a1, a2 = interval1
    b1, b2 = interval2

    # Calculate the start and end of the overlap interval
    start = max(a1, b1)
    end = min(a2, b2)

    # Check if there is an actual overlap
    if start <= end:
        return (start, end)
    else:
        return None  # No overlap

def overlap_percentage(interval1, interval2):
    b1, b2 = interval2
    overlap = find_overlap(interval1, interval2)
    if overlap == None:
        return 0
    else:
        (c1, c2) = overlap
        # Overlap as fraction of interval 2 (b)
        fraction = (c2 - c1) / (b2 - b1) * 100
        return fraction 
    
def log_overlap_percentage(interval1, interval2):
    return overlap_percentage(np.log(interval1), np.log(interval2))

with open('simulate.sh', 'w') as f:
    f.write('#!/bin/bash\n# Simulation script automatically generated by simulation-driver.ipynb, use this to create variants of it\n')
    f.write('rm -rf data\n')
    for instr in instrs[3:-6]:
        prec_type = instr.prec_type
        if instr.prec_type == 'wsp':
            prec_type = 'iwsp'

        # if instr.prec_type != 'foil':
            # continue
        
        print("-------------------------\n"+str(instr))
        for R in Rs:
            t = thickness_map[(instr.L0 * 1e10, R)]
            mode = 'GPU'
            # if instr.prec_type == 'foil':
            #     mode = 'CPU'

            # By = 0.01
            
            d_min, d_max = instr.delta_range()

            # print(f"delta_y range: {round(d_min * 1e9,2)} - {round(d_max * 1e9, 2)}nm")
            # print(f"\tRange of interest for R = {R * 1e9}nm: 0 - {3 * R * 1e9}")
            print(f"\tR = {R * 1e9}nm:")
            print(f"\t\tRange of interest:  {round(0.1 * R * 1e9)} - {round(3 * R * 1e9)}nm")

            

            overlap = find_overlap((d_min, d_max), (0.0, 3 * R))
            if overlap == None:
                print("No overlap!")
                f.write(f'echo "Skipping {instr.id}_{int(R * 1e10)} due to non-overlapping ranges!\n')

                raise Exception("No overlap!")
            else:
                (a,b) = overlap
                fraction = (b - a) / (3 * R) * 100
                log_fraction = log_overlap_percentage((d_min, d_max), (0.1 * R, 3 * R))
                print(f"\t\tOverlap: {round(a * 1e9,2)} - {round(b * 1e9, 2)}nm ({round(fraction,1)}% of linear range, {round(log_fraction,1)}% of log range)")
                max_ratio = b / d_max
                By_min = instr.By_min
                By_max = instr.By_max * max_ratio
                sim_cmd = f"./full-simulation.sh {N} 4 {N_steps} {By_min},{By_max} {instr.L0 * 1e10} {instr.DL* 1e10} {R * 1e10} {t} {instr.theta_0} {prec_type} {prec_type}_empty {mode}; rm -rf data_{instr.id}_{int(R * 1e10)}; mv data data_{instr.id}_{int(R * 1e10)}"
                # print(sim_cmd)
                f.write(f'{sim_cmd}\n')


In [ ]:
import pygad

target_interval = (20e-9, 5000e-9)

def normalize_solution(instance):    
    # L_max = 20.0 # m
    d_1 = 0.3 # m
    d_2 = 0.3 # m
    epsilon = 0.01 # m
    # Swap if the precession component lengths are the wrong way around
    if instance[2] < instance[3]:
        t = instance[2]
        instance[2] = instance[3]
        instance[3] = t
    min_dist = (d_1 + d_2) / 2 + epsilon
    prec_dist = instance[2] - instance[3]
    if prec_dist <= min_dist:
        delta = min_dist - prec_dist
        instance[2] += delta/2
        instance[3] -= delta/2
    # Limit the sample position to be straight after the second precession device with some margin given by epsilon
    instance[1] = min(instance[1], instance[3] - d_2 / 2 - epsilon)
    for i in range(len(instance)):
        instance[i] = round(instance[i], 4)

    return instance

def optimize_instrument(type='wsp', PG=True, L_min = 1.5, L_max = 10):
    # If a PG is used, change permitted L0 range
    if PG:
        quality = 0.01
        # Parameters are L0, L_s, L_1, L_2
        param_space = [{'low': 2.0, 'high': 4.4}, {'low': L_min, 'high': L_max}, {'low': L_min, 'high': L_max}, {'low': L_min, 'high': L_max}]
        monochrom_name = 'pyroletic graphite'
    else:
        quality = 0.1
        param_space = [{'low': 8.0, 'high': 12.0}, {'low': L_min, 'high': L_max}, {'low': L_min, 'high': L_max}, {'low': L_min, 'high': L_max}]
        monochrom_name = 'velocity selector'

    def instrument_from_solution(solution, type='wsp'):
        L0 = solution[0] * 1e-10
        L_s = solution[1]
        L_1 = solution[2]
        L_2 = solution[3]
        match type:
            case 'wsp':
                theta_0 = np.deg2rad(45)
                By_min = 0.1e-3
                By_max = 63e-3
            case 'iso':
                theta_0 = np.deg2rad(45)
                By_min = 0.1e-3
                By_max = 15e-3
            case 'foil':
                theta_0 = tune_foil(L0)
                By_min = 0.3e-3
                By_max = 30e-3
        instr = Instrument('', type, L0, quality * L0 / FWHM_factor, theta_0, By_min, By_max, L_s, L_1, L_2)
        return instr

    # Define the fitness function
    def fitness_func(solution):
        instr = instrument_from_solution(solution, type)
        delta_range = instr.delta_range()
        fitness = log_overlap_percentage(delta_range, target_interval) / 100.0 + overlap_percentage(delta_range, target_interval) / 1000.0
        return fitness
    N = 50000
    N_genes = 4
    # Generate the random array
    population = np.zeros((N, N_genes))

    for i, param in enumerate(param_space):
        population[:, i] = np.random.uniform(low=param['low'], high=param['high'], size=N)
    fitnesses = np.zeros(N)
    for j in range(0,N):
        population[j,:] = normalize_solution(population[j,:])
        fitnesses[j] = fitness_func(population[j,:])
    # print(fitnesses)
    best_id = np.argmax(fitnesses)
    best_sol = population[best_id, :]
    best_instr = instrument_from_solution(best_sol, type)

    best_instr.id = f'Best {type} instrument using {monochrom_name}'
    return best_instr

for type in ['foil', 'wsp', 'iso']:
    print(f"==========Best {type} instrument compatible with pyroletic graphite monochromator==========")
    pg_instr = optimize_instrument(type=type, PG = True, L_max=3.5)
    delta_range = pg_instr.delta_range()
    print(str(pg_instr))
    print(f"==========Best {type} instrument compatible with velocity selector monochromator==========")
    pg_instr = optimize_instrument(type=type, PG = False, L_max=3.5)
    delta_range = pg_instr.delta_range()
    print(str(pg_instr))

# Solve the equation
$$R_{pixel} = \frac{\sin(\pi p f)}{\pi p f} = 0.75$$

In [ ]:
from sympy import sin, nsolve, Symbol, pi
x = Symbol('x')

V_reduction = 0.7
solution = float(nsolve(sin(x) / x  - V_reduction, 0.4))
x_range = np.linspace(-4 * np.pi, 4 * np.pi, 10000)
y = np.sin(x_range) / x_range
plt.plot(x_range, y)
plt.axvline(solution, linestyle='--')
plt.axhline(V_reduction, linestyle='--')
plt.grid()
# solution

In [ ]:
f_min = solution / (np.pi * detector_pixel_size)
f_min, f_min * 2e-10 * 1.6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

optimal_min = []
optimal_max = []

num_samples = 1  # Number of samples to average
L_maxes = np.linspace(3.0, 9.0, 12)
for L_max in L_maxes:
    delta_ranges = []
    for _ in range(num_samples):
        pg_instr = optimize_instrument(PG=False, L_max=L_max)
        delta_range = np.array(pg_instr.delta_range())
        delta_ranges.append(delta_range)
    
    # Calculate the average delta_range over all samples
    avg_delta_range = np.max(delta_ranges, axis=0)
    
    optimal_min.append(avg_delta_range[0])
    optimal_max.append(avg_delta_range[1])


In [ ]:
plt.title(r'$\delta_{max}$ as function of instrument length')
# plt.plot(L_maxes, optimal_min, label=r'$\delta_{min}$')
plt.plot(L_maxes, np.array(optimal_max) * 1e9, '.', label='Wollaston prism')
plt.xlabel(r'$L_{1,max}$ [m]')
plt.ylabel(r'$\delta_{max}$ [nm]')
plt.legend()
plt.grid()
# plt.axhline(5e-6, linestyle='--', color='red')
plt.show()

In [ ]:
delta_range = pg_instr.delta_range()
# length_penalty = 0.01 * L_1 / L_1_0

fitness = log_overlap_percentage(delta_range, target_interval) 
np.log(delta_range), np.log(target_interval)
print(fitness)
# a,b = np.log(delta_range)
# a,b

# Milk sample (from dairy paper)

In [ ]:
delta_rho = 2.0e14 # 1/m^2 (?)
phi = 0.036
R = 50e-9
t = 2e-3
for (L0, _) in sources:
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    print(st)